In [15]:
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adadelta, Adagrad, Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, TerminateOnNaN
from keras import backend as K
from keras.losses import mean_squared_error
from keras.models import load_model, Model
import tensorflow as tf

# Normalize the data.
from sklearn import preprocessing
from keras.regularizers import l1_l2
import scipy.io as spio
import random

def pass_arg(nsim, tr_size, dropoutrate):
    print("Tr_size:", tr_size)
    def fix_seeds(seed):
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    #     K.set_session(sess)
        tf.compat.v1.keras.backend.set_session(sess)

    ss = 1
    fix_seeds(ss)

    # MC dropout
    class MCDropout(Dropout):
        def call(self, inputs, training=None):
            return super(MCDropout, self).call(inputs, training=True)


    # import pickle

    # def save_obj(obj, name):
    #     with open(name, 'wb') as f:
    #         pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

    # Compute the RMSE given the ground truth (y_true) and the predictions(y_pred)
    def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

    def PGNN_train_test(optimizer_name, optimizer_val, drop_rate, iteration, n_layers,
                        n_nodes, tr_size, pre_train, use_YPhy, lake_name):

        # Hyper-parameters of the training process
    #     batch_size = int(tr_size/2)
        batch_size = 1000
        num_epochs = 1000
        val_frac = 0.2
        patience_val = 100

        # Initializing results filename
        exp_name = "Upd" + optimizer_name + '_drop' + str(drop_rate) + '_nL' + str(n_layers) + '_nN' + str(n_nodes) + '_trsize' + str(tr_size) + '_iter' + str(iteration)
        exp_name = exp_name.replace('.','pt')
        results_dir = '../../results/Lake/'
        model_name = results_dir + exp_name + '.h5' # storing the trained model
        results_name = results_dir + exp_name + '_results.dat' # storing the results of the model

        # Load features (Xc) and target values (Y)
        data_dir = '../../data/'
        filename = lake_name + '.mat'
        mat = spio.loadmat(data_dir + filename, squeeze_me=True,
        variable_names=['Y','Xc_doy','Modeled_temp'])
        Xc = mat['Xc_doy']
        Y = mat['Y']

        # train and test data
        trainX, trainY = Xc[:tr_size,:], Y[:tr_size]
        testX, testY = Xc[-50:,:], Y[-50:]

#         # Loading unsupervised data
#         unsup_filename = lake_name + '_sampled.mat'
#         unsup_mat = spio.loadmat(data_dir+unsup_filename, squeeze_me=True,
#         variable_names=['Xc_doy1','Xc_doy2'])

#         uX1 = unsup_mat['Xc_doy1'] # Xc at depth i for every pair of consecutive depth values
#         uX2 = unsup_mat['Xc_doy2'] # Xc at depth i + 1 for every pair of consecutive depth values
    
#         if use_YPhy == 0:
#             # Removing the last column from uX (corresponding to Y_PHY)
#             uX1 = uX1[:,:-1]
#             uX2 = uX2[:,:-1]
            
        dependencies = {'root_mean_squared_error': root_mean_squared_error}

        # load the pre-trained model using non-calibrated physics-based model predictions (./data/unlabeled.dat)
        loaded_model = load_model(results_dir + pre_train, custom_objects=dependencies)

        # Creating the model
        model = Sequential()
        for layer in np.arange(n_layers):
            if layer == 0:
                model.add(Dense(n_nodes, activation='relu', input_shape=(np.shape(trainX)[1],)))
            else:
                model.add(Dense(n_nodes, activation='relu', kernel_regularizer=l1_l2(l1=.00, l2=.00)))
            # model.add(Dropout(rate=drop_rate))
            model.add(MCDropout(rate=drop_rate))
        model.add(Dense(1, activation='linear'))

        # pass the weights to all layers but 1st input layer, whose dimensions are updated
        for new_layer, layer in zip(model.layers[1:], loaded_model.layers[1:]):
            new_layer.set_weights(layer.get_weights())

        model.compile(loss='mean_squared_error',
                      optimizer=optimizer_val,
                      metrics=[root_mean_squared_error])

        early_stopping = EarlyStopping(monitor='val_loss', patience=patience_val,verbose=1)

        print('Running...' + optimizer_name)
        history = model.fit(trainX, trainY,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            verbose=1,
                            validation_split=val_frac, callbacks=[early_stopping, TerminateOnNaN()])

        test_score = model.evaluate(testX, testY, verbose=1)
        print(test_score)

            
        test_scores = []
        for i in range(int(nsim)):
#             print("simulation num:",i)
#             predictions = model.predict(testX)
#             samples.append(predictions)
            test_score = model.evaluate(testX, testY, verbose=0)
            test_scores.append(test_score[1])
        return np.array(test_scores)
    


    # Main Function
    if __name__ == '__main__':

        fix_seeds(1)

        # List of optimizers to choose from    
        optimizer_names = ['Adagrad', 'Adadelta', 'Adam', 'Nadam', 'RMSprop', 'SGD', 'NSGD']
        optimizer_vals = [Adagrad(clipnorm=1), Adadelta(clipnorm=1), Adam(clipnorm=1), Nadam(clipnorm=1), RMSprop(clipnorm=1), SGD(clipnorm=1.), SGD(clipnorm=1, nesterov=True)]

        # selecting the optimizer
        optimizer_num = 2
        optimizer_name = optimizer_names[optimizer_num]
        optimizer_val = optimizer_vals[optimizer_num]

        # Selecting Other Hyper-parameters
        drop_rate = dropoutrate # Fraction of nodes to be dropped out
        n_layers = 2 # Number of hidden layers
        n_nodes = 15 # Number of nodes per hidden layer

        
        # pre-trained model
        pre_train = 'Lake_Pre-trainAdam_drop0pt1_nL2_nN15_trsize300000_iter0.h5'
        tr_size = int(tr_size)

        # total number of runs
        iter_range = np.arange(1)
        
        #List of lakes to choose from
        lake = ['mendota' , 'mille_lacs']
        lake_num = 0  # 0 : mendota , 1 : mille_lacs
        lake_name = lake[lake_num]
        
        use_YPhy = 0
        
        testrmse=[]
        # iterating through all possible params
        for iteration in iter_range:
            # results, result_file, pred, obs, rmse = PGNN_train_test(optimizer_name, optimizer_val, drop_rate, 
                            # iteration, n_layers, n_nodes, tr_size, lamda, reg)
            # testrmse.append(rmse)
            pred = PGNN_train_test(optimizer_name, optimizer_val, drop_rate, 
                        iteration, n_layers, n_nodes, tr_size, pre_train, use_YPhy, lake_name)
    
    return np.squeeze(pred)


In [16]:
mean_rmses=[]
std_rmses=[]
for ii in ([13450]):
    test_rmse = pass_arg(50, ii, 0.1)
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)
    mean_rmses.append(mean_rmse)
    std_rmses.append(std_rmse)

Tr_size: 13450
Running...Adam
Train on 10760 samples, validate on 2690 samples
Epoch 1/1000
10760/10760 [==============================] - 0s 12us/step - loss: 216.0880 - root_mean_squared_error: 13.4408 - val_loss: 189.0938 - val_root_mean_squared_error: 12.0297
Epoch 2/1000
10760/10760 [==============================] - 0s 3us/step - loss: 185.8661 - root_mean_squared_error: 12.2435 - val_loss: 161.7162 - val_root_mean_squared_error: 10.8718
Epoch 3/1000
10760/10760 [==============================] - 0s 2us/step - loss: 152.8054 - root_mean_squared_error: 10.8856 - val_loss: 128.4274 - val_root_mean_squared_error: 9.5524
Epoch 4/1000
10760/10760 [==============================] - 0s 2us/step - loss: 118.8495 - root_mean_squared_error: 9.4406 - val_loss: 99.5990 - val_root_mean_squared_error: 8.4450
Epoch 5/1000
10760/10760 [==============================] - 0s 2us/step - loss: 89.5746 - root_mean_squared_error: 8.0645 - val_loss: 77.4151 - val_root_mean_squared_error: 7.4252
Epoch 6/

10760/10760 [==============================] - 0s 2us/step - loss: 10.0207 - root_mean_squared_error: 2.3875 - val_loss: 9.6376 - val_root_mean_squared_error: 2.3273
Epoch 46/1000
10760/10760 [==============================] - 0s 2us/step - loss: 10.0560 - root_mean_squared_error: 2.4003 - val_loss: 9.7154 - val_root_mean_squared_error: 2.3464
Epoch 47/1000
10760/10760 [==============================] - 0s 2us/step - loss: 10.1438 - root_mean_squared_error: 2.3901 - val_loss: 9.9242 - val_root_mean_squared_error: 2.4053
Epoch 48/1000
10760/10760 [==============================] - 0s 2us/step - loss: 9.8283 - root_mean_squared_error: 2.3825 - val_loss: 9.1793 - val_root_mean_squared_error: 2.2735
Epoch 49/1000
10760/10760 [==============================] - 0s 2us/step - loss: 10.1275 - root_mean_squared_error: 2.3754 - val_loss: 8.7888 - val_root_mean_squared_error: 2.2507
Epoch 50/1000
10760/10760 [==============================] - 0s 2us/step - loss: 10.0815 - root_mean_squared_error:

10760/10760 [==============================] - 0s 2us/step - loss: 8.7684 - root_mean_squared_error: 2.2159 - val_loss: 8.6438 - val_root_mean_squared_error: 2.1409
Epoch 91/1000
10760/10760 [==============================] - 0s 2us/step - loss: 8.8804 - root_mean_squared_error: 2.2383 - val_loss: 9.0668 - val_root_mean_squared_error: 2.2407
Epoch 92/1000
10760/10760 [==============================] - 0s 2us/step - loss: 8.9720 - root_mean_squared_error: 2.2426 - val_loss: 8.5694 - val_root_mean_squared_error: 2.1625
Epoch 93/1000
10760/10760 [==============================] - 0s 2us/step - loss: 8.9803 - root_mean_squared_error: 2.2552 - val_loss: 8.2895 - val_root_mean_squared_error: 2.1539
Epoch 94/1000
10760/10760 [==============================] - 0s 2us/step - loss: 8.7941 - root_mean_squared_error: 2.2239 - val_loss: 8.4515 - val_root_mean_squared_error: 2.1967
Epoch 95/1000
10760/10760 [==============================] - 0s 2us/step - loss: 8.6320 - root_mean_squared_error: 2.19

10760/10760 [==============================] - 0s 2us/step - loss: 8.5651 - root_mean_squared_error: 2.1713 - val_loss: 8.0914 - val_root_mean_squared_error: 2.0769
Epoch 136/1000
10760/10760 [==============================] - 0s 2us/step - loss: 8.4514 - root_mean_squared_error: 2.1563 - val_loss: 8.2849 - val_root_mean_squared_error: 2.0870
Epoch 137/1000
10760/10760 [==============================] - 0s 2us/step - loss: 7.9977 - root_mean_squared_error: 2.1199 - val_loss: 8.2513 - val_root_mean_squared_error: 2.1231
Epoch 138/1000
10760/10760 [==============================] - 0s 2us/step - loss: 7.9746 - root_mean_squared_error: 2.1227 - val_loss: 7.3494 - val_root_mean_squared_error: 2.0192
Epoch 139/1000
10760/10760 [==============================] - 0s 2us/step - loss: 8.1592 - root_mean_squared_error: 2.1485 - val_loss: 7.7251 - val_root_mean_squared_error: 2.0557
Epoch 140/1000
10760/10760 [==============================] - 0s 2us/step - loss: 7.9533 - root_mean_squared_error:

Epoch 180/1000
10760/10760 [==============================] - 0s 2us/step - loss: 7.9252 - root_mean_squared_error: 2.0721 - val_loss: 7.6283 - val_root_mean_squared_error: 2.0037
Epoch 181/1000
10760/10760 [==============================] - 0s 2us/step - loss: 7.7693 - root_mean_squared_error: 2.0863 - val_loss: 7.9902 - val_root_mean_squared_error: 2.0380
Epoch 182/1000
10760/10760 [==============================] - 0s 2us/step - loss: 7.8454 - root_mean_squared_error: 2.0668 - val_loss: 8.1377 - val_root_mean_squared_error: 2.0771
Epoch 183/1000
10760/10760 [==============================] - 0s 2us/step - loss: 7.9012 - root_mean_squared_error: 2.0830 - val_loss: 7.8750 - val_root_mean_squared_error: 2.0168
Epoch 184/1000
10760/10760 [==============================] - 0s 2us/step - loss: 7.6741 - root_mean_squared_error: 2.0701 - val_loss: 7.1682 - val_root_mean_squared_error: 1.9519
Epoch 185/1000
10760/10760 [==============================] - 0s 2us/step - loss: 7.7248 - root_mean

...

10760/10760 [==============================] - 0s 2us/step - loss: 4.1234 - root_mean_squared_error: 1.4901 - val_loss: 4.5865 - val_root_mean_squared_error: 1.5542
Epoch 808/1000
10760/10760 [==============================] - 0s 2us/step - loss: 4.1381 - root_mean_squared_error: 1.4865 - val_loss: 4.6226 - val_root_mean_squared_error: 1.5572
Epoch 809/1000
10760/10760 [==============================] - 0s 2us/step - loss: 4.1000 - root_mean_squared_error: 1.4857 - val_loss: 4.4586 - val_root_mean_squared_error: 1.5152
Epoch 810/1000
10760/10760 [==============================] - 0s 2us/step - loss: 4.0290 - root_mean_squared_error: 1.4790 - val_loss: 4.5216 - val_root_mean_squared_error: 1.5567
Epoch 811/1000
10760/10760 [==============================] - 0s 2us/step - loss: 4.0793 - root_mean_squared_error: 1.4876 - val_loss: 4.4606 - val_root_mean_squared_error: 1.5210
Epoch 812/1000
10760/10760 [==============================] - 0s 2us/step - loss: 4.0931 - root_mean_squared_error:

10760/10760 [==============================] - 0s 2us/step - loss: 3.9141 - root_mean_squared_error: 1.4567 - val_loss: 4.6126 - val_root_mean_squared_error: 1.5874
Epoch 852/1000
10760/10760 [==============================] - 0s 1us/step - loss: 3.9409 - root_mean_squared_error: 1.4733 - val_loss: 4.3650 - val_root_mean_squared_error: 1.5312
Epoch 853/1000
10760/10760 [==============================] - 0s 2us/step - loss: 3.9069 - root_mean_squared_error: 1.4700 - val_loss: 4.4550 - val_root_mean_squared_error: 1.5379
Epoch 854/1000
10760/10760 [==============================] - 0s 1us/step - loss: 3.8313 - root_mean_squared_error: 1.4392 - val_loss: 4.5202 - val_root_mean_squared_error: 1.5611
Epoch 855/1000
10760/10760 [==============================] - 0s 3us/step - loss: 3.8958 - root_mean_squared_error: 1.4589 - val_loss: 4.2903 - val_root_mean_squared_error: 1.5096
Epoch 856/1000
10760/10760 [==============================] - 0s 1us/step - loss: 3.7822 - root_mean_squared_error:

10760/10760 [==============================] - 0s 2us/step - loss: 3.8102 - root_mean_squared_error: 1.4356 - val_loss: 4.3869 - val_root_mean_squared_error: 1.5190
Epoch 897/1000
10760/10760 [==============================] - 0s 2us/step - loss: 3.8299 - root_mean_squared_error: 1.4339 - val_loss: 4.1447 - val_root_mean_squared_error: 1.4811
Epoch 898/1000
10760/10760 [==============================] - 0s 2us/step - loss: 3.7872 - root_mean_squared_error: 1.4342 - val_loss: 4.5012 - val_root_mean_squared_error: 1.5434
Epoch 899/1000
10760/10760 [==============================] - 0s 1us/step - loss: 3.7643 - root_mean_squared_error: 1.4196 - val_loss: 4.2181 - val_root_mean_squared_error: 1.5117
Epoch 900/1000
10760/10760 [==============================] - 0s 1us/step - loss: 3.7942 - root_mean_squared_error: 1.4273 - val_loss: 4.4572 - val_root_mean_squared_error: 1.5288
Epoch 901/1000
10760/10760 [==============================] - 0s 2us/step - loss: 3.6580 - root_mean_squared_error:

10760/10760 [==============================] - 0s 2us/step - loss: 3.5395 - root_mean_squared_error: 1.3978 - val_loss: 4.5490 - val_root_mean_squared_error: 1.5509
Epoch 942/1000
10760/10760 [==============================] - 0s 2us/step - loss: 3.6401 - root_mean_squared_error: 1.4131 - val_loss: 3.9453 - val_root_mean_squared_error: 1.4742
Epoch 943/1000
10760/10760 [==============================] - 0s 1us/step - loss: 3.6558 - root_mean_squared_error: 1.4045 - val_loss: 4.6499 - val_root_mean_squared_error: 1.5621
Epoch 944/1000
10760/10760 [==============================] - 0s 3us/step - loss: 3.7998 - root_mean_squared_error: 1.4303 - val_loss: 4.5860 - val_root_mean_squared_error: 1.5284
Epoch 945/1000
10760/10760 [==============================] - 0s 2us/step - loss: 3.7090 - root_mean_squared_error: 1.4246 - val_loss: 4.4563 - val_root_mean_squared_error: 1.5408
Epoch 946/1000
10760/10760 [==============================] - ETA: 0s - loss: 3.3568 - root_mean_squared_error: 1.3

Epoch 986/1000
10760/10760 [==============================] - 0s 1us/step - loss: 3.5882 - root_mean_squared_error: 1.4001 - val_loss: 4.1846 - val_root_mean_squared_error: 1.5117
Epoch 987/1000
10760/10760 [==============================] - 0s 3us/step - loss: 3.5882 - root_mean_squared_error: 1.4032 - val_loss: 4.2058 - val_root_mean_squared_error: 1.4937
Epoch 988/1000
10760/10760 [==============================] - 0s 1us/step - loss: 3.5588 - root_mean_squared_error: 1.3853 - val_loss: 4.0850 - val_root_mean_squared_error: 1.4636
Epoch 989/1000
10760/10760 [==============================] - 0s 3us/step - loss: 3.5862 - root_mean_squared_error: 1.3846 - val_loss: 4.1985 - val_root_mean_squared_error: 1.5174
Epoch 990/1000
10760/10760 [==============================] - 0s 1us/step - loss: 3.4626 - root_mean_squared_error: 1.3750 - val_loss: 4.3201 - val_root_mean_squared_error: 1.5197
Epoch 991/1000
10760/10760 [==============================] - 0s 1us/step - loss: 3.6459 - root_mean

In [12]:
mean_rmses

[1.5202339506149292]

In [6]:
std_rmses

[0.2463927147977416]

In [5]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mean_rmses, "../mean_rmse_dnn_upd_MC.dat")
save_obj(std_rmses, "../std_rmse_dnn_upd_MC.dat")

In [3]:
mc_pred=np.mean(pred,axis=0)
mc_pred

array([ 2.62061860e-02,  1.87012106e-02,  2.03988180e-02,  1.67454164e-02,
        1.15877548e-02,  1.77968144e-02, -1.27896736e-03, -1.31999478e-02,
        2.60994807e-02,  1.33960219e-02,  2.77881604e-02,  9.34297452e-04,
        6.04256568e-03,  1.26129938e-02,  7.02638784e-03,  2.18978971e-02,
        2.32310016e-02, -4.19802684e-03,  1.63644198e-02,  2.18827017e-02,
        2.80083790e-02,  2.09308118e-02,  2.16732565e-02,  1.84855182e-02,
        1.77870262e-02,  3.32493819e-02,  1.94747355e-02, -9.24617343e-04,
       -1.61748938e-02,  4.68686111e-02,  1.53396726e-02,  3.43751386e-02,
        1.31887319e-02,  2.40315683e-02,  2.57186349e-02,  1.02430042e-02,
        4.46724612e-03,  7.93455448e-03,  3.40867825e-02,  1.84419602e-02,
        9.89243481e-03,  2.23036781e-02,  2.42859274e-02,  2.76430584e-02,
        2.40111295e-02,  1.80043541e-02,  1.87582728e-02,  2.57925484e-02,
        1.78488772e-02,  2.23268848e-02, -3.57775800e-02,  1.67228747e-02,
        2.13348549e-02,  

In [4]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mc_pred, "../pred_upd_MC_Xx.dat")